<a href="https://colab.research.google.com/github/DarthJkid/FBREF_Analysis/blob/main/Top_Five_Leagues_Player_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Packages

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import statsmodels.api as sm

# Data Scraping

In [7]:
df = pd.read_html('https://fbref.com/en/comps/Big5/stats/players/Big-5-European-Leagues-Stats', attrs={'id': 'stats_standard'}) [0]
df.head()

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Unnamed: 3_level_0  \
                  Rk             Player             Nation                Pos   
0                  1         Max Aarons            eng ENG                 DF   
1                  2         Max Aarons            eng ENG                 MF   
2                  3     Rodrigo Abajas             es ESP                 DF   
3                  4     James Abankwah             ie IRL              DF,MF   
4                  5  Keyliane Abdallah             fr FRA                 FW   

  Unnamed: 4_level_0  Unnamed: 5_level_0 Unnamed: 6_level_0  \
               Squad                Comp                Age   
0        Bournemouth  eng Premier League             25-026   
1           Valencia          es La Liga             25-026   
2           Valencia          es La Liga             21-263   
3            Udinese          it Serie A             21-014   
4          Marseille          fr Ligue 1             18-300   

  Unnamed: 7_level_0 Playing Time         ... Per 90 Minutes              \
                Born           MP Starts  ...            Ast   G+A  G-PK   
0               2000            3      1  ...           0.00  0.00  0.00   
1               2000            1      0  ...           0.00  0.00  0.00   
2               2003            1      1  ...           0.00  0.00  0.00   
3               2004            6      0  ...           0.00  0.00  0.00   
4               2006            1      0  ...           0.00  0.00  0.00   

                                           Unnamed: 37_level_0  
  G+A-PK    xG   xAG xG+xAG  npxG npxG+xAG             Matches  
0   0.00  0.00  0.00   0.00  0.00     0.00             Matches  
1   0.00  0.00  0.00   0.00  0.00     0.00             Matches  
2   0.00  0.10  0.00   0.10  0.10     0.10             Matches  
3   0.00  0.06  0.00   0.06  0.06     0.06             Matches  
4   0.00  0.00  0.00   0.00  0.00     0.00             Matches  

[5 rows x 38 columns]

# Data Cleaning and Preprocessing
